In [25]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 

In [43]:
data = pd.read_csv("2.csv")


In [45]:
# todo ['home_team','away_team','country','country_full_home','country_full_away']
# 这些特征处理一下
data_without_label = data.drop(columns = ['score_diff','home_won',
'home_team','away_team','country','country_full_home','country_full_away'
])
labels = data['home_won']+1

# labels
SPLIT_LEN = 4500
train_data = data_without_label.iloc[:SPLIT_LEN]
test_data = data_without_label.iloc[SPLIT_LEN:]

train_labels = labels.iloc[:SPLIT_LEN]
test_labels = labels.iloc[SPLIT_LEN:]

# train_ds
BATCH_SIZE = 64
train_data =  tf.data.Dataset.from_tensor_slices(train_data)
train_label = tf.data.Dataset.from_tensor_slices(train_labels)

train_ds = tf.data.Dataset.zip((train_data, train_label)).batch(BATCH_SIZE)

test_data = tf.data.Dataset.from_tensor_slices(test_data)
test_label  = tf.data.Dataset.from_tensor_slices(test_labels)

test_ds = tf.data.Dataset.zip((test_data, test_label)).batch(BATCH_SIZE)
# list(train_ds.as_numpy_iterator())

In [46]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 24), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [47]:
test_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 24), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [35]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [36]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = Dense(24,activation='relu')
        self.d2 = Dense(128,activation='relu')
        self.d3 = Dense(3)

    def call(self,x):
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

model = MyModel()

In [37]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [38]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [39]:
@tf.function
def train_step(datas, labels):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(datas, training=True)
        loss = loss_object(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [40]:
@tf.function
def test_step(datas, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(datas, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [50]:
for data,labels in test_ds:
    print(data)
    print(labels)

    test_step(data, labels)

tf.Tensor(
[[ 1.50739200e+09 -2.41500997e+00  2.03818593e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 1.50739200e+09  7.29296661e-01 -5.39264475e-01 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 1.50747840e+09 -3.35442093e-01  1.04685885e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 ...
 [ 1.52933760e+09 -7.34719125e-01  1.52600027e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 1.52942400e+09 -6.01626781e-01  1.93244437e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.52942400e+09 -8.67811469e-01  1.81678955e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]], shape=(64, 24), dtype=float64)
tf.Tensor(
[2 1 0 0 2 2 0 2 1 0 0 2 2 1 0 2 2 1 2 2 2 0 0 2 2 2 2 2 0 2 0 2 1 1 1 1 1
 1 1 1 1 1 1 0 2 2 2 2 0 1 2 0 2 1 0 0 1 0 2 2 0 0 2 2], shape=(64,), dtype=int64)
tf.Tensor(
[[ 1.52951040e+09 -3.85351722e-01  1.08981636e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 [ 1.52951040e+09 -5.26208613e-02  2

In [52]:
EPOCHS = 50

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for data, labels in train_ds:
        train_step(data, labels)

    for data, test_labels in test_ds:
        test_step(data, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 1.0328762531280518, Accuracy: 50.37778091430664, Test Loss: 1.0590633153915405, Test Accuracy: 46.11819076538086
Epoch 2, Loss: 1.0328763723373413, Accuracy: 50.37778091430664, Test Loss: 1.0590633153915405, Test Accuracy: 46.11819076538086
Epoch 3, Loss: 1.0328762531280518, Accuracy: 50.37778091430664, Test Loss: 1.0590633153915405, Test Accuracy: 46.11819076538086
Epoch 4, Loss: 1.0328763723373413, Accuracy: 50.37778091430664, Test Loss: 1.059063196182251, Test Accuracy: 46.11819076538086
Epoch 5, Loss: 1.0328763723373413, Accuracy: 50.37778091430664, Test Loss: 1.059063196182251, Test Accuracy: 46.11819076538086
Epoch 6, Loss: 1.0328763723373413, Accuracy: 50.37778091430664, Test Loss: 1.0590630769729614, Test Accuracy: 46.11819076538086
Epoch 7, Loss: 1.0328763723373413, Accuracy: 50.37778091430664, Test Loss: 1.059063196182251, Test Accuracy: 46.11819076538086
Epoch 8, Loss: 1.0328764915466309, Accuracy: 50.37778091430664, Test Loss: 1.0590630769729614, Test Accurac